# Exploración de datos

In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.functions import lit

## Carga de tablas

In [2]:
spark = SparkSession.builder.getOrCreate()

23/10/03 19:02:58 WARN Utils: Your hostname, chidalgo-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.64 instead (on interface wlp0s20f3)
23/10/03 19:02:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/03 19:02:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/03 19:02:59 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# Cargamos la base de datos de balances y la revisamos en su estructura
balances = spark.read.csv('../data/archive/balances.csv', header=True)
balances.show(5)

+-----+--------------------+------+---------+--------+--------------+------------+--------+
|month|                  ID|  type|  product|  entity|balance_amount|days_default|  period|
+-----+--------------------+------+---------+--------+--------------+------------+--------+
|   11|c844ef92b32ec94ba...|type_7|product_2|entity_2|        3.4051|         5.0|period_5|
|   10|c817cb3a37a6b6403...|type_7|product_2|entity_1|        3.4003|         5.0|period_5|
|    6|1d1f726d844c5a343...|type_7|product_2|entity_5|        3.0592|         5.0|period_5|
|    6|7c77a3fd230db1698...|type_7|product_2|entity_2|        2.8013|         5.0|period_5|
|   11|a8702fb4fbdcb7356...|type_7|product_2|entity_2|         2.985|         5.0|period_5|
+-----+--------------------+------+---------+--------+--------------+------------+--------+


In [4]:
# Cargamos la base de datos customers y la visualizamos
customers = spark.read.csv('../data/archive/customers.csv', header=True)
customers.show(5)

+--------------------+------+------+---------------------+--------+-----------+-------+---------+---------+---------+---------+-------+-------+-------+
|                  ID|   age|income|time_from_specialized|type_job|bureau_risk|payroll|product_1|product_2|product_3|product_4|ofert_1|ofert_2|ofert_3|
+--------------------+------+------+---------------------+--------+-----------+-------+---------+---------+---------+---------+-------+-------+-------+
|42af3bf64c2c14307...|4.2484|4.1259|                  0.0|  type_2| category_4| 4.8095|      Yes|       No|       No|      Yes|     No|     No|     No|
|5ab9acd6dc30e0e93...|4.6903| 3.957|                  0.0|  type_7| category_9|    0.0|       No|       No|       No|      Yes|     No|     No|     No|
|8e40f988dd93417a0...|4.1682| 4.464|                  0.0|  type_7| category_5|    0.0|       No|       No|       No|      Yes|     No|     No|     No|
|0e48974ce6e81d9e8...|3.9651|4.0966|                  0.0|  type_7| category_3|    0.0| 

In [5]:
# Cargamos la base liabilities y la visualizamos
liabilities = spark.read.csv('../data/archive/liabilities.csv', header=True)
liabilities.show(5)

+-----+--------------------+---------+---------+--------+
|month|                  ID|product_1|product_2|  period|
+-----+--------------------+---------+---------+--------+
|   11|c7a6cecac361a970e...|     4.19|      0.0|period_5|
|   12|68f89dc7efe7cd1ce...|   4.4621|      0.0|period_5|
|    2|e3f6afb18792d0b97...|   4.2878|      0.0|period_5|
|    1|8993170400939d522...|      5.0|      5.0|period_5|
|    7|399d686897232a55c...|   3.8922|      0.0|period_5|
+-----+--------------------+---------+---------+--------+


In [6]:
# Se carga la base de datos movements y se visualiza
movements = spark.read.csv('../data/bbva-data-challenge-2023/archive/movements.csv', header=True)
movements.show(5)

+--------+--------------------+-----+------+------+------+------+
|  period|                  ID|month|type_1|type_2|type_3|type_4|
+--------+--------------------+-----+------+------+------+------+
|period_9|a66d4bbb5278c8a58...|   12|   0.0|   0.0|   0.0|3.9437|
|period_9|0d0a4fb4f1c98add6...|    1|3.7397|3.3499|   0.0|   0.0|
|period_9|a5b9c4d7ae3dfe5cc...|    4|3.6273|3.9726|3.6655|4.4131|
|period_9|8a583965c827c3776...|    8|3.7855|4.8286|   5.0|4.5094|
|period_9|e53bf4249c7d6d573...|    7|2.6077|3.9344|1.3229|3.8061|
+--------+--------------------+-----+------+------+------+------+


In [7]:
# Cargamos universe train y visualizamos (añadimos etiqueta)
universe_train = spark.read.csv('../data/bbva-data-challenge-2023/archive/universe_train.csv', header=True)
universe_train = universe_train.withColumn("partition", lit("train"))
universe_train.show(5)

+--------------------+---------+--------+---------+
|                  ID|attrition|  period|partition|
+--------------------+---------+--------+---------+
|4d5a33701cb5b30b0...|        0|period_5|    train|
|bf7092e5f394d2661...|        0|period_5|    train|
|270d8f8f607d19886...|        0|period_5|    train|
|5e6b7bea5e4911329...|        0|period_5|    train|
|98954adf775b9fce1...|        0|period_5|    train|
+--------------------+---------+--------+---------+


In [8]:
# Cargamos universe test, homologamos con train y visualizamos
universe_test = spark.read.csv('../data/bbva-data-challenge-2023/archive/universe_test.csv', header=True)
universe_test = universe_test.withColumn('attrition',  lit(np.nan))
universe_test = universe_test.withColumn('partition',  lit("test"))
universe_test.show(5)

+--------------------+--------+---------+---------+
|                  ID|  period|attrition|partition|
+--------------------+--------+---------+---------+
|20b427c65c62ee017...|period_9|      NaN|     test|
|4881d4a2146002a02...|period_9|      NaN|     test|
|28ff12351f215091b...|period_9|      NaN|     test|
|5cd2c0dc92c1c5ffa...|period_9|      NaN|     test|
|ceebdd2bbaafd83d8...|period_9|      NaN|     test|
+--------------------+--------+---------+---------+


In [9]:
# unimos test y train
universe = universe_train.unionByName(universe_test)

In [10]:
# finalmente, revisamos el ejemplo de cómo debe ser el output
sample_submission = spark.read.csv('../data/bbva-data-challenge-2023/archive/sample_submission.csv', header=True)
sample_submission.show(5)

+--------------------+---------+--------+---------+
|                  ID|attrition|  period|partition|
+--------------------+---------+--------+---------+
|4d5a33701cb5b30b0...|        0|period_5|    train|
|bf7092e5f394d2661...|        0|period_5|    train|
|270d8f8f607d19886...|        0|period_5|    train|
|5e6b7bea5e4911329...|        0|period_5|    train|
|98954adf775b9fce1...|        0|period_5|    train|
+--------------------+---------+--------+---------+


## Unión entre tablas

3697738